In [61]:
import json
from random import choice
import copy
import random

entity_path = "../entity/"
csv_path = "../csv/"
train_path = "../train/"
intent_path = "../intent/"

import csv

def read_csv(file, column):
    reader = csv.reader(open(csv_path + file, 'r', encoding='utf-8'), delimiter="\t")
    result = []
    for i in reader:
        result.append(i[column-1])
        
    return result

def load_json(path, file):
    with open(path + file + ".json", 'r', encoding='utf-8') as jsonfile:
        return json.load(jsonfile)

def load_entity(file):
    return load_json(entity_path, file)

def load_intent(file):
    return load_json(intent_path, file)

samples_dict = {}


def find_entity(entities, obj_name):
    for i in entities:
        if i['entity'] == obj_name:
            return i

def handle_enum_type(entity):
    e = entity['enum']
    name = entity['entity']
    if name not in samples_dict:
        samples_dict[name] = read_csv(e['source'], e['column'])
        
    return len(samples_dict[name]), entity, []
        
def handle_compound_type(js, entity):
    children = []
    total_number = 0
    for i in entity['compound']:
        c, sample, tp = handle_entity(js, i['type'])
        total_number = max(c, total_number)
        children.append((i['name'], sample, tp))
        
    return total_number, entity, children
        
def handle_choice_type(js, entity):
    children = []
    total_number = 0
    for i in entity['choice']:
        c = handle_entity(js, i)
        children.append(c)
        total_number += c[0]
        
    return total_number, entity, children   

def handle_entity(js, c):
    entity = find_entity(js, c)
    if 'enum' in entity:
        return handle_enum_type(entity)
    elif 'compound' in entity:
        return handle_compound_type(js, entity)
    elif 'choice' in entity:
        return handle_choice_type(js, entity)
    
    print("ERROR", entity)
    return 0,0,0



def get_generated_number_of_samples(total, n, allocated):
    number = (n * allocated) // total
    return max(10, number)

def generate_enum_samples(entity, n_samples):
    name = entity['entity']
    samples = [choice(samples_dict[name]) for i in range(n_samples)]
    return name, samples, 'enum'

def split_pattern(pattern):
    pos = pattern.find('@{')
    if pos < 0:
        return [pattern]
    
    end = pattern.find('}')
    
    result = []
    if pos > 0:
        result += [pattern[:pos]]
        
    result += [pattern[pos:end+1]]
    
    if end < len(pattern) - 1:
        result += split_pattern(pattern[end+1:])
    
    return  result

def get_pattern(pattern):
    result = split_pattern(pattern)
    
    tags = {}
    for s, i in zip(result, range(len(result))):
        if s.startswith('@{'):
            tags[s] = i
    
    return tags, result


def get_pos(sample, index):
    start = 0
    for i in range(index):
        start += len(sample[i])
        
    return start, len(sample[index])
        
def generate_compound_samples(entity, children, n_samples):
    samples = []

    for child in children:
        name = child[0]
        this_entity = child[1]
        this_children = child[2]
        if 'enum' in this_entity:
            _, sample, tp = generate_enum_samples(this_entity, n_samples)
            samples.append((name, sample, tp))
        elif 'compound' in this_entity:
            _, sample, tp = flatten_generate_compound_samples(this_entity, this_children, n_samples)
            samples.append((name, sample, tp))
        elif 'choice' in this_entity:
            _, sample, tp = flatten_generate_choice_samples(this_entity, this_children, n_samples)
            samples.append((name, sample, tp))
    
    #print("samples: ", samples, n_samples)
    
    mandatory_dict = {}
    for ent in entity['compound']:
        #print(ent)
        mandatory_dict[ent['name']] = ent['mandatory']
    
    patterns = entity['patterns']
    all_samples = []
    #print(n_samples)
    for p in patterns:
        pattern = get_pattern(p)
        #print(pattern)
        
        for i in range(n_samples):
            s = copy.deepcopy(pattern[1])
            #print(s)
            #record = {}
            tags = copy.deepcopy(pattern[0])
           
            for sample_group in samples:
                sample_group_data = sample_group[1]
                tag = '@{' + sample_group[0] + '}'
                
                if tag in tags: 
                    #print("===: ", i, tags[tag])
                    select_data = sample_group_data[i]
                    if (not mandatory_dict[sample_group[0]]) and choice([False, True]):
                        select_data = ""
                    
                    s[tags[tag]] = select_data
                
            for k in tags:
                tags[k] = get_pos(s, tags[k])
            
            #print(str(len(s)) + " : " + str(n_samples))
            #print("ss: ", s, tags)
            all_samples.append((tags, ''.join(s)))
            
    
    random.shuffle(all_samples)
    all_samples = all_samples[:n_samples]

    #print(all_samples)
    return entity['entity'], all_samples, 'compound'
    
def flatten_generate_compound_samples(entity, children, n_samples):
    e, samples, _ = generate_compound_samples(entity, children, n_samples)
    return entity['entity'], [i[1] for i in samples], 'enum'
    
def flatten_samples(entity, samples):
    results = []
    for s in samples:
        #print(s)
        typ = s[2]
        if typ == 'compound':
            results += [i[1] for i in s[1]]
        elif typ == 'enum':
            results += s[1]
        else:
            print("ERROR")
            
    return entity, results, 'enum'
    
def flatten_generate_choice_samples(entity, children, n_samples):
    e, samples, _ = generate_choice_samples(entity, children, n_samples)
    return flatten_samples(e, samples)
            

def generate_choice_samples(entity, children, n_samples):
    all_samples = []
    for i in sorted(children, key=lambda x: x[0]):
        s = (n_samples // len(children)) + 1 #get_generated_number_of_samples(n, i[0], m)
        
        ent = i[1]
        if 'enum' in ent:
            r = generate_enum_samples(ent, s)
        elif 'compound' in ent:
            r = flatten_generate_compound_samples(ent, i[2], s)
        elif 'choice' in ent:
            r = flatten_generate_choice_samples(ent, s)
        else:
            r = None
        
        all_samples.append(r)

    return entity['entity'], all_samples, 'choice'
    

def generate_sample(tree, n_samples):
    entry = tree[1]
    children = tree[2]
    if 'choice' in entry:
        return generate_choice_samples(entry, children, n_samples)
    elif 'compound' in entry:
        return generate_compound_samples(entry, children, n_samples)
    elif 'enum' in entry:
        return generate_enum_samples(entry, n_samples)
    
    print("ERROR")
    
    return []

def make_enum_label(samples, label):
    result = []
    for sample in samples:
        s = []
        for i in sample:
            s.append([i, label])
        result.append(s)
        
    return result

def make_compound_label(samples, label):
    result = []
    for sample in samples:
        s = []
        for i in sample[1]:
            s.append([i, label])
        result.append(s)
        
    return result

def generated_labeled_choice_sample(samples):
    result = []
    for i in range(len(samples)):
        label = "C" + str(i)
        ss = samples[i]
        if ss[2] == 'enum':
            result += make_enum_label(ss[1], label)
        elif ss[2] == 'compound':
            result += make_compound_label(ss[1], label)
            
    return result


def make_enum_test_data(samples):
    result = []
    for sample in samples:
        s = []
        for i in sample:
            s.append([i])
        s.append(["。"])
        result.append(s)
        
    return result
    
def make_compound_test_label(samples):
    result = []
    for sample in samples:
        s = []
        for i in sample[1]:
            s.append([i])
        s.append(["。"])
        result.append(s)
        
    return result
    
def generate_test_data(samples):
    result = []
    for i in range(len(samples)):
        ss = samples[i]
        if ss[2] == 'enum':
            result += make_enum_test_data(ss[1])
        elif ss[2] == 'compound':
            result += make_compound_test_label(ss[1])
            
    return result

range_tags = ["A", "B","C","D","E","F","G","H"]

def generate_compound_labels(entity):
    labels = {}
    
    print(entity)
    elems = entity['compound']
    
    for n, i in zip(elems, range(len(elems))):
        labels["@{" + n["name"] + "}"] = range_tags[i]
        
    return labels

def label_compound_samples(entity, samples):
    results = []
    
    labels = generate_compound_labels(entity)
    
    #print(samples)
    for i in samples:
        label = i[0]
        data = [[c, 'O'] for c in i[1]]

        for tag in label:
            begin, size = label[tag]
            #print("---", begin, size)
            for k in range(begin, begin+size):
                data[k][1] = labels[tag]

        data += [['。', 'O']]
        
        results += data
        
    
    #print(data)
    return results

def label_choice_samples(samples):
    results = []
    
    print(samples)
    for s, i in zip(samples, range(len(samples))):
        for d in s[1]:
            results += [[c, range_tags[i]] for c in d]
            results += [['。' , 'O']]
    
    return results

def do_write_csv_data(entity, samples, ty):
    with open(train_path + entity + "/" + ty + '.data', 'w', encoding='utf-8') as csvfile:
        my_writer = csv.writer(csvfile, delimiter='\t',lineterminator='\n')
        for i in samples:
            my_writer.writerow(i)
            
def do_write_training_data(entity, samples):
    print(samples)
    do_write_csv_data(entity, samples, "train")

def do_write_test_data(entity, samples):
    do_write_csv_data(entity, samples, "test")

        
def write_training_data(js, samples):
    entity = samples[0]
    typ = samples[2]
    data = samples[1]
    random.shuffle(data)

    if typ == 'compound':
        do_write_training_data(entity, label_compound_samples(find_entity(js, entity), data))
    elif typ == 'choice':
        do_write_training_data(entity, label_choice_samples(data))

def make_samples(js, entity_name, num):
    return generate_sample(handle_entity(js, entity_name), num)
    
def make_training_file(js, entity_name, num):
    write_training_data(js, make_samples(js, entity_name, num))

def make_compound_test_samples(samples):
    results = []
    #print(samples)
    for i in samples:
        results += i[1]
        results.append(['。'])

    return results

def make_choice_test_samples(samples):
    results = []
    
    for s in samples:
        for d in s[1]:
            results += [[c] for c in d]
            results += [['。']]
    
    return results


def write_test_data(samples):
    entity = samples[0]
    typ = samples[2]
    data = samples[1]
    random.shuffle(data)
    
    if typ == 'compound':
        do_write_test_data(entity, make_compound_test_samples(data))
    elif typ == 'choice':
        do_write_test_data(entity, make_choice_test_samples(data))

def make_test_file(js, entity_name, num):
    write_test_data(make_samples(js, entity_name, num))

def make_entity_data(js, entity_name, train_n=100, test_n=10):
    make_training_file(js, entity_name, train_n)
    make_test_file(js, entity_name, test_n)
    
    
js = load_entity('datetime') + load_entity('date') + load_entity('time') 
js += load_entity('city') + load_entity('ticket')
js += load_intent('book_ticket')
#result
#get_pattern("从@{from}到@{end}sh")

In [5]:
make_entity_data(js, "datetime")

In [63]:
make_entity_data(js, "time", 1000)

[('range-time', ['早晨十点五十四分左右到中午九点三十分左右', '从傍晚二十点五十四分左右到上午一点四十八分左右', '晚上二十二点四十七分左右至凌晨十一点二十分左右之间', '从早晨三点五十八分左右到二十三点零六分', '中午八点五十四分左右至中午八点十七分之间', '凌晨五点五十八分左右到中午六点四十六分', '中午十六点零三分至十四点三十五分之间', '下午十八点左右到十一点四十一分', '六点整到早晨十点四十一分左右', '从下午零点三十三分到六点二十四分', '从早晨十点左右到中午七点一刻左右', '傍晚十九点四十一分至傍晚十点五十一分之间', '从凌晨三点零七分左右到上午十点四十九分', '凌晨三点零七分左右至上午十点四十九分之间', '下午十四点四十二分到下午四点零二分', '早晨一点十九分到晚上零点二十八分左右', '从二十一点五十二分到晚上二十三点二十五分', '从傍晚六点二十三分到早晨五点十三分', '凌晨七点五十五分到下午二十三点二十分左右', '下午十二点三十九分左右至中午十四点三十四分之间', '傍晚十点三十五分左右到上午十六点五十三分', '早晨一点零八分至上午三点二十七分之间', '八点零二分至上午九点二十五分之间', '傍晚十四点二十七分到中午二十二点四十七分', '晚上二十三点三十二分到晚上二十一点零五分左右', '凌晨十二点五十二分至上午九点二十二分之间', '凌晨二十三点一刻至上午十五点十四分之间', '晚上十一点左右至傍晚零点半之间', '从傍晚六点四十五分左右到早晨十八点五十七分左右', '从下午十四点三十八分到中午二十三点四十一分', '下午十一点三十一分到晚上十七点五十三分', '从中午十六点三十四分左右到十六点二十一分', '凌晨四点五十分至早晨九点五十分左右之间', '傍晚十一点零五分左右至上午六点三十三分左右之间', '从五点二十一分到下午七点五十六分', '从九点四十一分到早晨二十点零二分', '从晚上六点四十二分左右到晚上九点二十分', '下午二十三点十五分左右至下午三点零七分之间', '十点十九分到中午十八点二十七分', '从晚上八点四十六分到中午六点三十一分左右', '凌晨七点五十五分至下午二十三点二十分左右之间', '傍晚十三点五十三分左右至下午十八点十六分左右之间', '中午七点整左右

In [3]:
make_entity_data(js, "range-time", 100)

In [12]:
make_entity_data(js, "general-city", 100)

In [45]:
make_entity_data(js, "book_ticket", 100)

{'entity': 'book_ticket', 'compound': [{'name': 'datetime', 'type': 'datetime', 'mandatory': True}, {'name': 'from', 'type': 'general-city', 'mandatory': False}, {'name': 'to', 'type': 'general-city', 'mandatory': True}, {'name': 'ticket', 'type': 'ticket', 'mandatory': True}], 'patterns': ['我想订@{datetime}@{from}到@{to}的@{ticket}', '帮我看看@{datetime}从@{from}到@{to}的@{ticket}', '查一下@{datetime}从@{from}到@{to}的@{ticket}', '查下@{datetime}到@{to}的@{ticket}', '问下@{datetime}去@{to}的@{ticket}', '查下@{datetime}从@{from}去@{to}的@{ticket}', '帮我看看到@{to}的@{ticket}的情况', '帮我看看到@{to}的@{ticket}还有吗', '帮我看看有没有@{datetime}去@{to}的@{ticket}', '@{datetime}@{from}到@{to}的@{ticket}', '@{datetime}去@{to}的@{ticket}']}
